<a href="https://colab.research.google.com/github/kanadn/DiHT-GCC/blob/main/Kanad's_Copy_of_AMLProjectTinkering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount Google drive, `cd` to the path

In [1]:
#mount Google drive and connect to a path
from google.colab import drive
import os

drive.mount('/content/drive')
%cd /content/drive/MyDrive/AdvancedML/AMLProject/diht

Mounted at /content/drive
/content/drive/MyDrive/AdvancedML/AMLProject/diht


Install requirements

In [ ]:
!pip install -r requirements.txt
!pip install -e .

For some reason we need a specific version of Pillow

In [3]:
!pip install Pillow==9.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 24.3 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
diht 1.0 requires pillow==9.4.0, but you have pillow 9.0.0 which is incompatible.


These models are available in the code...

In [ ]:
import diht
print(diht.available_models())

['diht_vitb32_224px', 'diht_vitl14_336px', 'diht_vitb16_224px']


In [ ]:
import torch
import diht

from diht import model_zoo
from PIL import Image


text_tokenizer, image_transform, model = model_zoo.load_model(
    "diht_vitl14_336px", is_train=False
)

image = Image.open("infer_image.png").convert("RGB")
image = image_transform(image).unsqueeze(0)
text_captions = ["a mountain", "a beach", "a desert"]
text = text_tokenizer(text_captions)

with torch.no_grad():
    image_features, text_features, logit_scale = model(image, text)
    logits_per_image = logit_scale * image_features @ text_features.T
    probs = logits_per_image.softmax(dim=-1).numpy()

print(f"text captions: {text_captions}")
print(f"text caption probs: {probs}")

Sample code is working, now let's do some tinkering...

In [ ]:
text_captions2 = ["a mountain", "a lake", "a mountain and a lake"]
texts = text_tokenizer(text_captions2)

with torch.no_grad():
    image_features, text_features, logit_scale = model(image, texts)
    logits_per_image = logit_scale * image_features @ text_features.T
    probs = logits_per_image.softmax(dim=-1).numpy()

print(f"text captions: {text_captions2}")
print(f"text caption probs: {probs}")

The model seems to be smart :)

Now let's import Google's [Conceptual Captions](https://ai.google.com/research/ConceptualCaptions/) dataset...  
Edit: Since the dataset only has image URLs, we need to fecth the images. This might take a lot of time, like, a lot!  
The following approach stores the dataset into a map.

In [4]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 10.6 MB/s eta 0:00:00


Found another way but we won't be able to use these actual images in drive, so nevermind: https://github.com/igorbrigadir/DownloadConceptualCaptions

In [5]:
from concurrent.futures import ThreadPoolExecutor
from functools import partial
import io
import json
import os

from typing import Callable, Tuple, Union

import PIL.Image
from torch.utils.data import Dataset
from datasets import load_dataset

from datasets.utils.file_utils import get_datasets_user_agent
import urllib
from torchvision.datasets.folder import find_classes, make_dataset

class ImageSingleCaptionDataset(Dataset):
    r"""Generic class for image-caption datasets where each image is associated with
        a single caption.
    Methods:
    --------
    __getitem__(index): Tuple
        Return a tuple (image, captions). Both are preprocessed with
        respective transforms.
    len(): int
        Return the number of samples.
    Attributes:
    number_of_captions: int
        Return the number of captions in the dataset.
    """

    def __init__(
        self,
        name: str,
        split: str,
        transform: Callable = None,
    ) -> None:
        r"""Constructor for ImageMultiCaptionDataset.
        Parameters:
        -----------
        split: str
            The split (train or val or test).
        transform: Callable
            An image transformation.
        Returns:
        --------
        None
        """
        self._name = name
        self._split = split
        self._transform = transform
        self._load_data()

    def fetch_single_image(self, image_url, timeout=None, retries=0):
        USER_AGENT = get_datasets_user_agent()
        for _ in range(retries + 1):
            try:
                request = urllib.request.Request(
                    image_url,
                    data=None,
                    headers={"user-agent": USER_AGENT},
                )
                with urllib.request.urlopen(request, timeout=timeout) as req:
                    image = PIL.Image.open(io.BytesIO(req.read()))
                break
            except Exception:
                image = None
        return image
    
    def fetch_images(self, batch, num_threads, timeout=None, retries=0):
        fetch_single_image_with_args = partial(self.fetch_single_image, timeout=timeout, retries=retries)
        with ThreadPoolExecutor(max_workers=num_threads) as executor:
            batch["image"] = list(executor.map(fetch_single_image_with_args, batch["image_url"]))
        
        return batch
    
    def _load_data(self) -> None:
        # Huggingface code to load the dataset
        dset = load_dataset("conceptual_captions")
        dset_val = dset[self._split].select(range(100)) # Select the first 100 examples in validation set
        
        num_threads = 20

        dset_val = dset_val.map(self.fetch_images, batched=True, batch_size=100, fn_kwargs={"num_threads": num_threads})

        processed_data = self._prepare_data_from_dict(dset_val)
        
        self._text = processed_data["captions"]
        self._txt2img = processed_data["txt2img"]
        self._img2txt = processed_data["img2txt"]
        self._data = processed_data["image_captions_data"]
        self._number_of_captions = int(sum(len(captions) for _, captions in self._data))

    def _prepare_data_from_dict(self, dataset):
        text = []
        txt2img = []
        img2txt = []
        image_captions_data = []
        img_count = 0
        # Here since each image has only one caption, we don't need to keep track of text_id and img_id
        for index, dset_dict in enumerate(dataset):
            if dset_dict["image"] is not None:
              image_i = dset_dict["image"]
              captions_i = dset_dict["caption"]
              text.append(dset_dict["caption"])
              txt2img.append(img_count)        
              img2txt.append(img_count)
              image_captions_data.append((image_i, captions_i))
              img_count += 1
            else:
              continue

        return {
            "captions": text,
            "txt2img": txt2img,
            "img2txt": img2txt,
            "image_captions_data": image_captions_data,
        }

    def __len__(self) -> int:
        r"""Return number of images in the dataset.
        Parameters
        ----------
        None
        Returns
        -------
        len: int
            Number of images in the dataset.
        """
        return len(self._data)

    @property
    def number_of_captions(self) -> int:
        r"""Return number of captions in the dataset.
        Parameters
        ----------
        None
        Returns
        -------
        number_of_captions: int
            Number of captions in the dataset.
        """
        return self._number_of_captions

    # This method is used by the dataloader to get the data, so keep it as it is
    def __getitem__(self, index: int) -> Union[Tuple, None]:
        r"""Return sample as tuple (image, index).
        We only return image and index because the captions could be of varying size
        and it could break default collates.
        The class implements _load_caption to handle loading captions.
        Parameters:
        -----------
        index: int
            The index of the sample.
        Returns:
        --------
        image: array_like
            The image for the `index` sample.
        index: int
            The index of the sample.
        Exception handling:
        -------------------
        If an exception is raised during dataloading,
        this function returns `None`.
        """
        image = self._load_image(index)
        return image, index

    def _load_image(self, index: int) -> Tuple:
      # TODO: Convert image to RGB
      image = self._data[index][0]
      image = image.convert("RGB")
      if self._transform is not None:
          image = self._transform(image)
      return image

    def _load_caption(self, index):
        caption = self._data[index][1]
        return caption

    # def _load_text(self, index: int) -> Tuple:
    #     captions = self._load_captions(index)
    #     return captions, self._img2txt[index]

    def _load_item(self, index: int) -> Tuple:
        image = self._load_image(index)
        captions = self._load_caption(index)
        return image, captions
    

class GCC(ImageSingleCaptionDataset):
    r"""The Google Conceptual Captions dataset.
    Webpage: https://ai.google.com/research/ConceptualCaptions/
    """

    def __init__(
        self,
        split: str,
        transform: Callable = None,
    ) -> None:
        r"""Constructor for GCC dataset.
        Parameters:
        -----------
        root: str
            Path where train and val split are saved.
        split: str
            The split (train or val or test).
        transform: Callable
            An image transformation.
        Returns:
        --------
        None
        """
        super().__init__(
            name="gcc",
            split=split,
            transform=transform,
        )

    def __repr__(self) -> str:
        return "\n".join(
            [
                "GCC(",
                f"  split={self._split},",
                f"  n_images={self.__len__()},",
                f"  n_captions={self.number_of_captions}"
                f"  transform={self._transform}",
                ")",
            ]
        )

In [6]:
import numpy as np

import torch
import torch.nn.functional as F

from diht import model_zoo

from torch.utils.data import DataLoader
from tqdm import tqdm

In [7]:
MODEL_NAME = "diht_vitl14_336px"  # replace with the model you want to evaluate
TEMPLATE = "a photo of {}."
  
# get tokenizer, transform, model, eval dataset and dataloader
print(f"Load model {MODEL_NAME} ...")
text_tokenizer, image_transform, model = model_zoo.load_model(
    MODEL_NAME, is_train=False)

Load model diht_vitl14_336px ...


100%|█████████████████████████████████████| 1.59G/1.59G [00:52<00:00, 32.5MiB/s]


In [8]:
gcc_dataset = GCC(split="validation", transform=image_transform)

Generating train split:   0%|          | 0/3318333 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/15840 [00:00<?, ? examples/s]

Dataset conceptual_captions downloaded and prepared to /root/.cache/huggingface/datasets/conceptual_captions/unlabeled/1.0.0/05266784888422e36944016874c44639bccb39069c2227435168ad8b02d600d8. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [9]:
gcc_dataloader = DataLoader(
    dataset=gcc_dataset,
    batch_size=32,
    num_workers=5,
)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# move model to GPU
model = model.cuda()

In [15]:
# Updated for GCC dataset
def extract_features(model, text_tokenizer, eval_dataset, eval_dataloader):
    all_image_features = []
    all_text_features = []
    with torch.no_grad():
        for batch in tqdm(eval_dataloader):
            images, image_idxs = batch[0], batch[1]
            image_features = model.encode_image(images)
            image_features = F.normalize(image_features, dim=-1)
            all_image_features.append(image_features.detach().cpu())
            for imidx in image_idxs.tolist():
                caption = eval_dataset._load_caption(imidx)
                caption = TEMPLATE.format(caption)
                caption = text_tokenizer(caption)
                text_features = model.encode_text(caption)
                text_features = F.normalize(text_features, dim=-1)
                all_text_features.append(text_features.detach().cpu())
    return (
        torch.cat(all_image_features),
        torch.cat(all_text_features),
    )

# compute image features
print("Extract features for GCC ...")
(
    gcc_image_features,
    gcc_text_features
) = extract_features(model, text_tokenizer, gcc_dataset, gcc_dataloader)

Extract features for GCC ...




  0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


 33%|███▎      | 1/3 [05:29<10:59, 329.52s/it]

 67%|██████▋   | 2/3 [11:07<05:34, 334.32s/it]

100%|██████████| 3/3 [13:54<00:00, 278.29s/it]


In [18]:
print(gcc_image_features.shape)
print(gcc_text_features.shape)

torch.Size([80, 768])
torch.Size([80, 768])


In [52]:
sim_i2t = gcc_image_features @ gcc_text_features.t()
sim_t2i = sim_i2t.t()
sim_i2t.shape

torch.Size([80, 80])

In [27]:
a = np.arange(10)
print(np.where(a < 5, a, 10*a))

[ 0  1  2  3  4 50 60 70 80 90]


In [30]:
inds = np.argsort([5, 6, 3, 1, 7, 9])[::-1]
inds

array([5, 4, 1, 0, 2, 3])

In [54]:
i = 55
tmp_arr = np.where(inds == i)[0]
tmp_arr

array([], dtype=int64)

In [49]:
def compute_recall(
    scores_i2t,
    scores_t2i,
    txt2img,
    img2txt,
    k=[1, 5, 10],
):
    """Compute retrieval recall metrics."""
    # Original images-to-text
    # ranks = np.zeros(scores_i2t.shape[0])
    # for index, score in enumerate(scores_i2t):
    #     inds = np.argsort(score)[::-1]
    #     # score
    #     rank = int(1e20)
    #     for i in img2txt:
    #         tmp_arr = np.where(inds == i)[0]
    #         tmp = tmp_arr[0]
    #         if tmp < rank:
    #             rank = tmp
    #     ranks[index] = rank
    # # overall: compute text retrieval recall
    # trs = {}
    # for topk in k:
    #     trs[topk] = len(np.where(ranks < topk)[0]) / len(ranks)


    # image-to-text
    ranks = np.zeros(scores_i2t.shape[0])
    for index, score in enumerate(scores_i2t):
        inds = np.argsort(score)[::-1]
        i2t_curr_rank_arr = np.where(inds == img2txt[index])[0]
        i2t_curr_rank = i2t_curr_rank_arr[0]
        ranks[index] = i2t_curr_rank
    print(f"ranks for image-to-text: {ranks}")
    # overall: compute image retrieval recall
    trs = {}
    for topk in k:
        trs[topk] = len(np.where(ranks < topk)[0]) / len(ranks)

    # text-to-image
    ranks = np.zeros(scores_t2i.shape[0])
    for index, score in enumerate(scores_t2i):
        inds = np.argsort(score)[::-1]
        t2i_curr_rank_arr = np.where(inds == txt2img[index])[0]
        t2i_curr_rank = t2i_curr_rank_arr[0]
        ranks[index] = t2i_curr_rank
    print(f"ranks for text-to-image: {ranks}")
    # overall: compute image retrieval recall
    irs = {}
    for topk in k:
        irs[topk] = len(np.where(ranks < topk)[0]) / len(ranks)
    return trs, irs

In [50]:
def evaluate_retrieval(image_features, text_features, txt2img, img2txt):
    # compute similarity: image2text (i2t) and text2image (t2i)
    sim_i2t = image_features @ text_features.t() # Will this work for single caption?
    sim_t2i = sim_i2t.t()
    # evaluate image-text matching (itm)
    i2t, t2i = compute_recall(
        sim_i2t.numpy(),
        sim_t2i.numpy(),
        txt2img,
        img2txt,
        k=[1],
    )
    return i2t, t2i

In [51]:
gcc_i2t, gcc_t2i = evaluate_retrieval(
        gcc_image_features,
        gcc_text_features,
        gcc_dataset._txt2img,
        gcc_dataset._img2txt,
    )

ranks for image-to-text: [ 5.  0.  0.  0.  0.  0.  0.  0. 29.  2.  0.  0.  0.  1.  0.  1. 32. 79.
 57.  0.  0.  4.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  1.  0.  0.  0.  0. 12.  0.  0.  0. 25.  5.  0.  0.  1.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0. 13.  0.  1.  0.  0.  0.  2.]
ranks for text-to-image: [10.  0.  0.  0.  0.  0.  0.  0. 34.  1.  1.  0.  0. 12.  0.  0. 35. 72.
 60.  0.  0.  1.  0.  0.  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  4.  0.
  0.  0.  4.  0.  0.  0.  0. 18.  0.  0.  0. 31. 22.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0. 29.  0.  1.  0.  5.  0.  4.]


In [48]:
print(f"GCC T2I r@1 for {MODEL_NAME}: {100*gcc_t2i[1]:.1f}")
print(f"GCC I2T r@1 for {MODEL_NAME}: {100*gcc_i2t[1]:.1f}")

GCC T2I r@1 for diht_vitl14_336px: 76.2
GCC I2T r@1 for diht_vitl14_336px: 77.5


In [ ]:
import matplotlib.pyplot as plt

num_images = 100
count = 0
# Select the first num_images from the validation set
validation_set = dset_val.select(range(num_images))

# Display each image and its caption
for i in range(num_images):
    image_url = validation_set[i]['image_url']
    image = validation_set[i]['image']
    caption = validation_set[i]['caption']
    if image != None:
      plt.imshow(image)
      count += 1
    plt.title(caption)
    plt.show()


In [ ]:
count

79

In [ ]:
import matplotlib.pyplot as plt

for i in range(5):
    image = dset_val[i]["image"] # Assuming 'image' is the key for the image data
    plt.imshow(image)
    plt.show()